<a href="https://colab.research.google.com/github/lucarubini/DeepLearning_quick_introduction/blob/main/demo_time_series/demo_time_series_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demo Serie Temporali (per previsione domanda prodotti)**


1.   Definizione delle funzioni e librerie da importare
2.   Caricamento e analisi dei dati
3.   Definizione e addestramento del modello
4.   Test 



### **Definizione delle funzioni e librerie da importare**

In [1]:
#Importare le librerie necessarie
import csv
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
def eval_test(test_df, filename, model, norm_vector,time_step=1):
  """
  funzione per generare un file di testo che contiene
  INPUT
   -test_df: DataFrame con contiene per ogni riga t la domanda di prodotto all'istante temporale t
   -filename: Nome del file di output in cui memorizzare gli output del modello
   -model: Modello per la stima della domanda
   -norm_vector: Vettore per la normalizazzione degli output
   -time_step(defaul=1): range temporale da prendere in considerazione per generare l'output al tempo t
  OUTPUT
   -f_out: file che contiene per ogni riga la stima della domanda di ciascun prodotto (l'output del modello)
  """
  n_examples = len(test_df)
  n_outputs = n_examples-time_step
  n_prods = int(train_max.size/2)
  print(f"Numero di prodotti: {str(n_prods)}")
  print(f"Numero di output da generare: {n_outputs}")
  print(f"Output filename: {filename}")
  with open(filename,'w') as f_out:
    for i in range(n_outputs):
      df_i = test_df[i:i+time_step]
      input_i = np.array(df_i)
      input_i = np.expand_dims(input_i, axis=0)
      out_i = model(input_i)
      out_i = out_i * train_max[n_prods:]
      f_out.write(" ".join([str(out_i[0,i].numpy()) for i in range(out_i.shape[1])])+"\n")

In [23]:
def avg_mse(filename,target_df):
  """
  funzione per calcolare l'MSE dat
  INPUT
   -filename: nome del file che contiene la previsione della richiesta di prodotti, una riga per timestep
   -target_df: target della richiesta di prodotti, una riga per timestep. Il formato e' Pandas DataFrame
  OUTPUT
   -mse_out: ("mse_$filename") file che contiene per ogni riga l'MSE di ciascun prodotto
   -mse_avg_out: ("mse_avg_$filename") file che contiene per ogni riga l'MSE medio di tutti i prodotti
  """
  avg_mse = 0
  with open(filename,'r') as f_in, open(f"mse_{filename}",'w') as mse_out, open(f"mse_avg_{filename}",'w') as mse_avg_out:
    for i,line in enumerate(f_in):
      tmp_i = [float(x) for x in line.rstrip().split()]
      tmp_out = np.array(tmp_i)
      out_i = np.array(tmp_out)
      tgt_i = np.array(target_df[i,:])
      mse_i = ((out_i - tgt_i)**2)
      mse_i_mean = mse_i.mean(axis=0)
      avg_mse += mse_i_mean
      mse_out.write(" ".join([str(mse_i[i]) for i in range(mse_i.shape[0])])+"\n")
      mse_avg_out.write(f"{mse_i_mean}"+"\n")

In [3]:
def test_avg_mse(filename):
  """
  funzione per calcolare MSE medio
  INPUT
   -filename: file che contiene l'MSE della stima della richiesta di tutti i prodotti, una riga per timestep
  OUTPUT
   -out: MSE medio di tutti gli MSE contenuti nel file $filename, e' la media di tutti gli errori medi fatti in test
  """
  out = 0
  with open(filename,'r') as f:
    for i, num in enumerate(f):
        out += float(num.rstrip())
  out = out / (i+1)
  return out

In [4]:
#Classe per generare le finestre di dati
class WindowGenerator():
  """
  Classe per gestire e generare le finestre di dati
  INPUT
   -input_width: time range per input
   -label_width: time range per target
   -shift: shift temporale 
   -train_df: DataFrame di Train
   -val_df: DataFrame di Valid
   -test_df: DataFrame di Test
   -input_columns: nome delle colonne usate per input
   -label_columns: nome delle colonne usate per target
  """
  def __init__(self,
               input_width,
               label_width,
               shift,
               train_df,
               val_df,
               test_df,
               input_columns=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.input_columns = input_columns
    if input_columns is not None:
      self.input_columns_indices = {name: i for i, name in
                                    enumerate(input_columns)}
    
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}
    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices (t): {self.input_indices}',
        f'Label indices (t): {self.label_indices}',
        f'Input column name(s): {self.input_columns}',
        f'Label column name(s): {self.label_columns}'])
  
  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.input_columns is not None:
      inputs = tf.stack(
          [inputs[:, :, self.column_indices[name]] for name in self.input_columns],
          axis=-1)
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)
    ds = ds.map(self.split_window)
    return ds

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result


In [5]:
def read_XY(filename, label, shift=0):
  """
  INPUT
   -filename: nome del file che contiene le domande per ciascun prodotto, una riga per prodotto, ogni riga sara' colonna del DataFrame di output. Possono essere sia i valori di input che quelli di output
   -label: nome delle colonne del DataFrame
   -shift(default=0): shift temporale. Le prime righe del DataFrame saranno shiftate da 'shift' righe che valgono 0. Lo shift e' temporale e serve per allineare/ritardare input e target
  OUTPUT
   -df: DataFrame che contiene i valori letti da $filename e riorganizzati
  """
  print(f"reading data shift: {str(shift)}")
  with open(filename,'rb') as f:
    for i, line in enumerate(f):
      tmp_line = line.decode('utf-8')
      tmp_values = [int(i) for i in tmp_line.strip().split(';') if i != '']
      shift_step = shift if shift is not 0 else None
      if shift_step is not None:
        tmp_head = [0 for _ in range(shift)]
        tmp_tail = tmp_values[:-shift]
        tmp_values = tmp_head + tmp_tail
      if i == 0:
        df = pd.DataFrame(data=tmp_values, columns=["{}".format(label[i])])
      df["{}".format(label[i])] = tmp_values
  print("Created DataFrame shape: {}".format(df.shape))
  return df

In [6]:
def compile_and_fit(model, window, patience=2):
  '''
  funzione per istanziare il modello e addestrarlo
  INPUT
   -model: modello tensorflow
   -window: oggetto per generare le finestre di dati
   -patiente(defaul=2): criterio per early stopping
  OUTPUT
   -history: modello addestrato
  ''' 
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train,
                      epochs=MAX_EPOCHS,
                      validation_data=window.val)
  return history

### **Caricamento e analisi dei dati**

Definiamo il numero di prodotti e i nomi delle colonne del DataFrame di INPUT e TARGET 

In [8]:
N_PRODS = 20
INPUT_LABELS = [f"prod_{i}" for i in range(N_PRODS)]
TARGET_LABELS = [f"target_{i}" for i in range(N_PRODS)]

In [ ]:
data_X = read_XY("X.txt",INPUT_LABELS,shift=0)
data_Y = read_XY("Y.txt",TARGET_LABELS,shift=1)

#DataFrame che unisce in un'unica tabella INPUT e OUTPUT
df = data_X.join(data_Y)

print(f"Numero di prodotti: {str(df.shape[1])}")
print(f"Numero di istanti temporali: {str(df.shape[0])}")

Osservare le prime righe del DataFrame

In [ ]:
df.head()

Vedere le statistiche di ciacun prodotto/target nel DataFrame

In [ ]:
df.describe().transpose()

In [ ]:
#Crea paritizion di train, val, dev

#Percentuale di suddivisione tra Train/Valid/Test (devono sommare a 1)
split_partition = [70,20,10]


column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*split_partition[0]/100)]
val_df = df[int(n*split_partition[0]/100):int(n*(split_partition[0]+split_partition[1])/100)]
test_df = df[int(n*(split_partition[0]+split_partition[1])/100):]

num_features = int(df.shape[1]/2)
num_products = int(df.shape[1]/2)
print(f"Numero di prodotti(input): {num_features}")
print(f"Numero di prodotti(target): {num_products}")
print(f"Numero esempi totali: {len(df)}")
print(f"Numero esempi train: {train_df.shape[0]}")
print(f"Numero esempi val: {val_df.shape[0]}")
print(f"Numero esempi test: {test_df.shape[0]}")

Calcoliamo ora il vettore di normalizzazione in base ai valori di training e poi normalizziamo i dataset di train/valid/test

In [13]:
#Normalizzare i valori
train_max = train_df.max()

train_df = train_df / train_max
val_df = val_df / train_max
test_df = test_df / train_max

### **Definizione e addestramento del modello**

Definiamo i parametri per l'addestramento del modello e la definizione delle finestre di dati.

*   MAX_EPOCHS: numero massimo di epoche
*   INPUT_time_STEP: finestra di input (quanti istanti t uso prima di generare output)
*   OUT_STEPS: finestra di output (quanti istanti temporali t di target voglio generare, nel nostro caso 1)




In [14]:
#Configurazione
MAX_EPOCHS = 20
INPUT_time_STEP=7
OUT_STEPS=1

Creiamo la Classe `multi_window` per generare le finestre di dati

In [ ]:
multi_window = WindowGenerator(input_width=INPUT_time_STEP,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               input_columns=[f"prod_{i}" for i in range(num_products)],
                               label_columns=[f"target_{i}" for i in range(num_products)])
print(multi_window)

Creiamo il modello `lstm_model` che ha


*   Layer LSTM di dimensione 32
*   Layer Dense di dimensione 50 (act='relu')
*   Layer Dense di output (output=numero di prodotti, funzione di attivazione relu per avere solo valori positivi)



In [16]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(50),
    tf.keras.layers.Dense(units=num_products, activation='relu')
])

Istanziamo e addestriamo il modello

In [ ]:
history = compile_and_fit(lstm_model, multi_window)

Visualizziamo un riassunto dell'architettura del modello

In [ ]:
lstm_model.summary()

Valutiamo le performance in test (ma e' fatto alle stesse condizioni del training, quindi con la stessa finestra temporale, e quindi poco utile).

Nella sezione successiva si implementa una pipeline di test migliore e piu' flessibile.

In [ ]:
eval_tf_test = lstm_model.evaluate(multi_window.test)

## **Test**

Questa pipeline di test e' costruita per generare i valori degli ultimi frame temporali del DataFrame `df` inserendo anche uno shfit temporale `time_step` quanti giorni prima utilizzo per crearmi il contesto.

Venongo poi estatti anche i valori di target del test per misurare l'MSE.


In [20]:
time_step = 10
norm_vector = train_max

input_label_test_df = df[-len(test_df)-time_step:] / norm_vector
input_label_test_df = input_label_test_df[INPUT_LABELS]
target_label_test_df = df[TARGET_LABELS][-len(test_df):].to_numpy()


I valori di output del modello sono gia' normalizzati

In [21]:
eval_test(input_label_test_df, 'output_32_50_test_10', lstm_model, norm_vector, time_step=time_step)

Numero di prodotti: 20
Numero di output da generare: 86
Output filename: output_32_50_test_10


In [24]:
avg_mse('output_32_50_test_10',target_label_test_df)

In [25]:
test_avg_mse('mse_avg_output_32_50_test_10')

10.119061617157648

## **Salvare e Caricare il modello**

In questa sezione vediamo come salavare un modello addestrato per poi inseguito caricarlo.

Ricordiamo che in un modello di Rete Neurale non è solo importante il valore dei pesi (le matrici), ma anche il grafo computazionale delle operazioni, percioà nella cartella in cui salveremo il modello 'MODEL_PATH', la struttura dati al suo interno è complessa.

In [ ]:
MODEL_PATH='model'
lstm_model.save(MODEL_PATH)

Vediamo ora come si carica un modello salvato: verrà ricostruitio il modello con pesi addestrati e grafo.

In [35]:
reconstructed_model = tf.keras.models.load_model(MODEL_PATH)

Calcoliamo l'output del modello appena caricato, quando poi andremo a confrontare làoutput di questo modello con quello generato dal modello salvato ci aspetteremo che i due file coincidano. 

In [ ]:
eval_test(input_label_test_df, 'output_reconstructed', reconstructed_model, norm_vector, time_step=time_step)

In [37]:
!diff output_32_50_test_10 output_reconstructed